In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
import db4
import pandas as pd
from datetime import datetime 

query="select * from stream_scraping_ids"
streams=db4.select(query)
daily_programs={}
for i in streams:
    # print(i['id'],i['stream_id'])
    print(f"{i['stream_name']} started in {datetime.now()}")

    # Configure Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  
    chrome_options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    stream_scrape_id=i['id']
    url=f"https://tv.mail.ru/baku/channel/{stream_scrape_id}/"

    driver.get(url)

    time.sleep(5)

    spans = driver.find_elements(By.CSS_SELECTOR, "div.filter__list.js-more__list span.filter__item.js-group_date.js-more__item")
    target_date = datetime.now().date()
    
    for span in spans:
        if span.get_attribute("data-value") == target_date:
            ActionChains(driver).move_to_element(span).click(span).perform()
            time.sleep(2)  
            break

    times = driver.find_elements(By.CLASS_NAME, "p-programms__item__time")
    names = driver.find_elements(By.CLASS_NAME, "p-programms__item__name")

    programs = {}
    for t, n in zip(times, names):
        programs[t.text] = n.text
        
    daily_programs[f"{i['stream_id']}"] = programs


    # for time, name in daily_programs.items():
    #     print(f"{time}: {name}")


    driver.quit()
        
    print(f"{i['stream_name']} finished in {datetime.now()}")



Xəzər TV started in 2024-12-02 11:28:38.383550
Xəzər TV finished in 2024-12-02 11:28:58.817037
İctimai Radio started in 2024-12-02 11:28:58.817037
İctimai Radio finished in 2024-12-02 11:29:17.334391
AzTV started in 2024-12-02 11:29:17.334391
AzTV finished in 2024-12-02 11:29:35.568035


In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime

def get_daily_program(streams):
    daily_programs = {}

    for i in streams:
        print(f"{i['stream_name']} started in {datetime.now()}")

        # Initialize driver and handle any setup errors
        try:
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--disable-gpu")
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        except Exception as e:
            print(f"Error initializing WebDriver for {i['stream_name']}: {e}")
            continue

        try:
            stream_scrape_id = i['id']
            url = f"https://tv.mail.ru/baku/channel/{stream_scrape_id}/"
            driver.get(url)
            time.sleep(5)
        except Exception as e:
            print(f"Error loading URL for {i['stream_name']}: {e}")
            driver.quit()
            continue

        try:
            spans = driver.find_elements(By.CSS_SELECTOR, "div.filter__list.js-more__list span.filter__item.js-group_date.js-more__item")
            target_date = datetime.now().strftime("%Y-%m-%d")

            date_found = False
            for span in spans:
                if span.get_attribute("data-value") == target_date:
                    ActionChains(driver).move_to_element(span).click(span).perform()
                    time.sleep(2)
                    date_found = True
                    break

            if not date_found:
                print(f"No data found for {i['stream_name']} on {target_date}. Moving to the next stream.")
                driver.quit()
                continue
        except Exception as e:
            print(f"Error interacting with date selection for {i['stream_name']}: {e}")
            driver.quit()
            continue

        try:
            times = driver.find_elements(By.CLASS_NAME, "p-programms__item__time")
            names = driver.find_elements(By.CLASS_NAME, "p-programms__item__name")

            if not times or not names:
                print(f"No program data available for {i['stream_name']} on {target_date}.")
                driver.quit()
                continue

            programs = {t.text: n.text for t, n in zip(times, names)}
            if programs:
                daily_programs[f"{i['stream_id']}"] = programs
        except Exception as e:
            print(f"Error extracting program data for {i['stream_name']}: {e}")
        finally:
            driver.quit()

        print(f"{i['stream_name']} finished in {datetime.now()}")

    return daily_programs

query="select * from stream_scraping_ids"
streams=db4.select(query)
if streams:
    daily_programs=get_daily_program(streams)
else:
    print('There are problem with streams')

Xəzər TV started in 2024-12-02 12:04:39.080672
Xəzər TV finished in 2024-12-02 12:04:58.247058
İctimai Radio started in 2024-12-02 12:04:58.247058
No data found for İctimai Radio on 2024-12-02. Moving to the next stream.
AzTV started in 2024-12-02 12:05:13.486833
AzTV finished in 2024-12-02 12:05:40.011879


In [ ]:
def clean_dict(dict):
    for key, value in dict.items():
        cleaned_value = value.split('.', 1)[0].strip()  
        cleaned_value = ' '.join(cleaned_value.split())  
        dict[key] = cleaned_value
    return dict
    
try:
    for i in daily_programs:
        daily_programs[i]= clean_dict(daily_programs[i])
except ValueError as e:
    print(e)

{'32': {'07:00': 'Kənd həyati',
  '07:30': 'Xəbər ertəsi. Прямая трансляция',
  '10:00': 'İncə səhər. Прямая трансляция',
  '12:00': 'Suna ilə yeməyə dəyər',
  '13:00': 'Xəzər xəbər',
  '13:20': 'Zindan Həyəti',
  '14:50': 'Xəzər xəbər',
  '15:15': 'Təsir dairəsi. Прямая трансляция',
  '18:15': 'Xəzər axşami. Прямая трансляция',
  '19:45': 'Azərlotereya. Прямая трансляция',
  '20:00': 'Xəzər axşami. Прямая трансляция',
  '21:00': 'Xəzər xəbər',
  '22:00': 'Xəyanət',
  '23:30': 'Bədii film'},
 '25': {'05:00': 'Görüş yeri',
  '06:00': 'Sənədli film',
  '07:00': 'Telesəhər',
  '10:00': 'Həftə. Analitik-informasiya proqramı',
  '12:00': 'AzTV Xəbər',
  '12:30': 'Günə davam',
  '14:00': 'AzTV Xəbər',
  '14:30': 'Güvənli sabah',
  '15:00': 'Paytaxt: Əbdülhəmid',
  '16:00': 'AzTV Xəbər',
  '17:00': 'Hədəf',
  '18:00': 'AzTV Xəbər',
  '18:30': 'Hədəf',
  '19:30': 'Parlament saatı',
  '20:00': 'AzTV Xəbər',
  '21:10': 'Hədəf',
  '23:00': 'Quruluş Osman',
  '00:00': 'AzTV Xəbər',
  '00:30': 'Bəd

In [35]:
def insert_code_to_programs(daily_programs):
    missing_dict = {}
    insert_programs_query = "INSERT INTO programs (name, stream_id) VALUES (%s, %s)"

    for stream in daily_programs:
        try:
            missing = set()
            select_programs_query = f"""SELECT * FROM programs WHERE stream_id={stream}"""
            df = db4.select(select_programs_query)

            for _, program_name in daily_programs[stream].items():
                if not any(program_name == record['name'] for record in df):
                    missing.add(program_name)

            missing_dict[stream] = missing
        except Exception as e:
            print(f"Error processing stream {stream}: {e}")
            continue

        try:
            for program_name in missing:
                db4.insert(insert_programs_query, (program_name, stream))
        except Exception as e:
            print(f"Error inserting program '{program_name}' for stream {stream}: {e}")

    return missing_dict
insert_code_to_programs(daily_programs)


{'32': set(), '25': set()}

In [37]:
def insert_code_to_tv_schedules(daily_programs):
    todays_date = datetime.now().date()
    target_date = todays_date.strftime('%Y-%m-%d')

    for stream in daily_programs:
        try:
            # Check if data already exists for the stream today
            query = f"SELECT * FROM tv_schedules WHERE stream_id={stream} AND start_time::DATE = '{todays_date}'"
            df = db4.select(query)
            
            if len(df) > 0:
                print(f"Pass this stream {stream}")
                continue

            print(f"Processing stream {stream} results")
            for start_time, program_name in daily_programs[stream].items():
                try:
                    start_datetime = datetime.strptime(f'{target_date} {start_time}', '%Y-%m-%d %H:%M')

                    insert_schedules = """
                        INSERT INTO tv_schedules (start_time, program_id, stream_id)
                        SELECT 
                            %s AS start_time, p.id AS program_id, %s AS stream_id
                        FROM programs p
                        WHERE p.name = %s
                        LIMIT 1;
                    """

                    values = (start_datetime, stream, program_name)
                    db4.insert(insert_schedules, values)
                except Exception as e:
                    print(f"Error inserting program '{program_name}' for stream {stream} at {start_time}: {e}")
        except Exception as e:
            print(f"Error processing stream {stream}: {e}")
insert_code_to_tv_schedules(daily_programs)


Pass this stream 32
Pass this stream 25


In [ ]:
missing_dict = {}
insert_programs_query = "INSERT INTO programs (name, stream_id) VALUES (%s, %s)"
for stream in daily_programs:
    missing = set()
    select_programs_query=f"""select * from programs where stream_id={stream}"""
    df=db4.select(select_programs_query)

    for a, b in daily_programs[stream].items():
        found = False
        for i in df:
            # print(i)
            if b == i['name']:
                found = True
                break
        if not found:
            missing.add(b)
            # print(b)
    for j in missing:
        # print(' ')
        db4.insert(insert_programs_query, (j, stream))
    missing_dict[stream]=missing
    

 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database
 
Connection to the PostgreSQL database


In [27]:
missing_dict

{'32': {'Bədii film'},
 '25': {'AzTV Xəbər',
  'Bədii film',
  'Görüş yeri',
  'Günə davam',
  'Güvənli sabah',
  'Hədəf',
  'Həftə',
  'Parlament saatı',
  'Paytaxt: Əbdülhəmid',
  'Quruluş Osman',
  'Sənədli film',
  'Telesəhər',
  'Xəbərlər'}}

In [ ]:
todays_date=datetime.now().date()
for stream in daily_programs:
    # print(stream)
    query=f"select * from tv_schedules where stream_id={stream} and start_time::DATE = '{todays_date}'"
    
    df=db4.select(query)
    if len(df)>0:
        print("Pass this stream",stream)
        continue    
    else:
        print(f"stream {stream} results")
        for a, b in daily_programs[stream].items():
            start_datetime = datetime.strptime(f'{target_date} {a}', '%Y-%m-%d %H:%M')
            
            insert_schedules = """
                INSERT INTO tv_schedules (start_time, program_id, stream_id)
                SELECT 
                    %s AS start_time, p.id AS program_id, %s AS stream_id
                FROM programs p
                WHERE p.name = %s
                LIMIT 1;
            """
            
            values = (start_datetime, stream, b)
            
            db4.insert(insert_schedules, values)
        

32
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
25
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connec

In [ ]:
sql = "INSERT INTO programs (name, stream_id) VALUES (%s, %s)"
for a, b in daily_programs.items():
    print(f"Inserting program: {b}")
    stream_id = 5  # Set the stream_id value
    db4.insert(sql, (b, stream_id))

In [ ]:
id_value = 2800
stream_id_value = 10


# Insert query
insert_query = """
INSERT INTO stream_scraping_ids (id, stream_id, stream_name)
SELECT 
    %s AS id, 
    %s AS stream_id, 
    s.full_name AS stream_name 
FROM streams s
WHERE s.id = %s;
"""

# Execute the insert
db4.insert(insert_query, (id_value, stream_id_value, stream_id_value))

In [ ]:
from datetime import datetime,timedelta
import db4

start_date = datetime.now().date()-timedelta(days=14)
query=f"""SELECT id,stream_id,program_id,youtube_link  FROM tv_schedules,youtube_scraping_days WHERE youtube_link IS NOT NULL and start_time>'{start_date}';"""
df=db4.select(query)

----------------------------------------------------------------------------
date = datetime.now().date()

for i in df:
    id=i['id']
    link=i['youtube_link']
    program_id=i['program_id']
    youtube_scraping_days=i['youtube_scraping_days']
    if youtube_scraping_days is None:
        youtube_scraping_days=1
    else:
        youtube_scraping_days+=1
    
    # print(link)
------------------------------------------------------------------------------------------------
update_query = "UPDATE tv_schedules SET program_id = %s, youtube_views=%s, youtube_likes=%s, youtube_comments=%s,youtube_scraping_days=%s  WHERE id = %s"

update_query=""
youtube_views =
youtube_likes =
youtube_comments =

db4.insert(update_query, (program_id, youtube_views,youtube_likes,youtube_comments,youtube_scraping_days,id))
------------------------------------------------------------------------------------
insert_query = """
INSERT INTO program_rates (schedule_id, youtube_views, youtube_likes, youtube_comments,program_id,rate_date)
VALUES (%s, %s, %s, %s,%s,%s);
"""
youtube_views =
youtube_likes =
youtube_comments =
# Execute the insert
db4.insert(insert_query, (schedule_id_value, youtube_views_value, youtube_likes_value, youtube_comments_value,program_id,date))

In [ ]:
date = datetime.now().date()-timedelta(days=3)
update_query = "UPDATE program_rates SET program_id = %s WHERE schedule_id = %s"
    
for i in tv:
    id=i['id']
    print(id)
    p_id=i['program_id']
    # print(date)
    db4.insert(update_query, (p_id, id)) 